##### Convention
## Testing During Interactive Programming Is Efficient

In interactive mode, the `testing` extension will execute tests when:
    
* An unassigned string literatal containing doctests.
* Docstrings in functions definitions and class defitions.
* Functions containing complete annotations.
* Classes containing runTest
* TestCase classes

In [1]:
    try:
        from .rites import Shell
    except:
        from rites import Shell

In [2]:
    from unittest import *
    from doctest import *
    from doctest import DocTestCase
    from ast import *
    from functools import wraps, partial
    from typing import Callable
    from dataclasses import dataclass, field
    from collections import Iterable
    from IPython import get_ipython
    from inspect import *
    from hypothesis import given, strategies, strategies as st, assume, HealthCheck, Verbosity, settings, find, errors
    __all__ = 'infer',
    
    CURRENT = None
    def infer(object):
        spec = getfullargspec(object)
        annotations = {}#{'return': None}
        annotations.update(spec.annotations) 
        returns = annotations.pop('return', None)
        if not spec.args: 
            return FunctionTestCase(object), annotations, returns
        
        main = __import__('__main__')
        for arg in spec.args:
            if arg in annotations: continue
            try:
                thing = eval(arg, vars(main))
            except NameError: continue
            if isinstance(thing, (type, list)):
                annotations[arg] = thing
                
        if not spec.defaults:
            
            try:
                annotations = {
                    str: st.from_type(object) if isinstance(object, type) and getattr(object, '__name__', '') != 'object'
                    else object if isinstance(object, st.SearchStrategy)
                    else st.one_of(list(map(st.just, object)))
                    for str, object in annotations.items()
                }
                if annotations:
                    return FunctionTestCase(given(**annotations)(object)), annotations.copy(), returns
            except: ...
        return None, annotations, returns
        


Allow iterable strategies, ghetto typed st

In [3]:
    from hypothesis import find

In [4]:
    @dataclass
    class Testing(Shell):
        examples = list()
        objects = list()
        any = False
        def post_run_cell(Testing):
            from types import ModuleType
            module, main, tests = ModuleType('__main__'), __import__('__main__'), list()
            module.__test__ = getattr(main, '__test__', {})
            while Testing.examples:
                tests.append(
                    DocTestCase(DocTest(
                        DocTestParser().get_examples(Testing.examples.pop(0)), main.__dict__, 
                        '__main__', '<testing>', 1, """""")))
            
            while Testing.objects:
                name = Testing.objects.pop(0)
                object = getattr(main, name)
                setattr(module, name, object)
                if isinstance(object, type):
                    if issubclass(object, TestCase):
                        tests.append(object())
                    elif hasattr(object, 'runTest'):
                        def runTest(): return object.runTest(object)
                        tests.append(FunctionTestCase(wraps(object.runTest)(runTest)))
                elif callable(object):
                    test, annotations, returns = infer(object)
                    if (returns is not None) and callable(returns) ^ isinstance(returns, type)  and len(annotations) is 1:
                        try:
                            find(*annotations.values(), lambda x: returns(object(x)))
                        except:
                            assert False, f"""NoSuchExample for {object} satifies {returns} """
                    else: test and tests.append(test)
            doctest = DocTestSuite(module, extraglobs=main.__dict__)
            doctest._tests and tests.append(doctest)
            try:
                if type(Testing).any:
                    tests and  TextTestRunner().run(TestSuite(tests))
            except: ...
            finally: type(Testing).any = False

In [5]:
    class DiscoverTests(NodeTransformer):
        def visit_Str(body, node):
            if '>>> ' in node.s:
                Testing.examples.append(node.s)
                Testing.any = True
            return node
            
        def visit_FunctionDef(body, node): 
            Testing.objects.append(node.name)
            Testing.any = True
            return node
        
        def visit_Assign(body, node): 
            Testing.any = True
            return node
        
        visit_ClassDef = visit_FunctionDef

In [6]:
    settings.register_profile('ip', settings(
        suppress_health_check=(HealthCheck.return_value,),
        verbosity=Verbosity.normal,))

In [7]:
    def load_ipython_extension(ip=get_ipython()):
        global testing, CURRENT
        unload_ipython_extension(ip)
        CURRENT = Testing()
        ip.ast_transformers = remove_discover()+[DiscoverTests()]
        ip.events.register('post_run_cell', CURRENT.post_run_cell)
        settings.load_profile('ip')

In [8]:
    def remove_discover(ip=get_ipython()):
        ip.ast_transformers = [_ for _ in ip.ast_transformers if not isinstance(_, DiscoverTests)]
        return ip.ast_transformers

In [9]:
    CURRENT = None
    def unload_ipython_extension(ip=get_ipython()):
        global CURRENT
        remove_discover()
        if CURRENT is not None:
            ip.events.unregister('post_run_cell', CURRENT.post_run_cell)

In [10]:
    if __name__ == '__main__':
        load_ipython_extension()

    !jupyter nbconvert --inplace --execute Untitled2.ipynb